# Define parent sequence class, later inherit class in a DNASeq and ProtienSeq class

 ### Sequence Class notes
    1.Sequence class mainly bulit from previous code

### DNASeq Class notes:
    1.DNASeq class dep
    requirements
        -a translate method that will convert the DNA sequence and return a ProteinSequence object
        -one other method of your choice (what you did previously is fine)
    2. DNA seq should check that it is in fact a DNA seq, functionality checking validity

### ProteinSeq Class notes:
            -a method of your choice. In this case, if the method you would implement is too complex to reasonably implement or would use resources you don't have access to, it is okay to leave it as what is called a stub method (has only one line, "pass") and explain in comments what this method would do and it's purpose
            a: ProtienSeq must mave different elements than DNASeq but similarties to Sequence class for inheir
            b: What might be unique to protien sequences that might be useful data to add to a method?

In [1]:
import datetime as dt#a way to snapshot current datetime

In [2]:
def readMultilineFASTA(file):

	fh = open(file)

	name = None
	seq = ""


	for line in fh:
		if line.startswith(">"):   #Dr. Whaley supplied fasta file pull in function
			if name:
				
				yield (name,seq)
				name = None
				seq = ""

			name = line.rstrip().lstrip(">")

		else:
			seq += line.rstrip()
			

	yield name,seq

#### parent class 

In [3]:
class Sequence:
    def __init__(self,stamp,seq): #initalize parent class Sequence
        self.seq = seq
        self.stamp = stamp
        self.orgLen = len(seq)
    def __repr__(self):
        return list(self.seq)

#### child class 1 of 2  (DNAsequence) checks for validity and provides a conversion to Protein Sequence
specific string return

In [4]:
class DNASeq(Sequence): # Child sequence class to harness DNA sequence record
    def __init__(self,stamp,seq,seqHeader,label,index):
      
        DNAbase=['G','A','T','C'] #Listto test valid DNASEQ
        failindex=0
        for i in range(len(seq)):
            if seq[i] not in DNAbase:  #Test bases for DNA validity
                failindex = i
        if failindex==0:
            super().__init__(stamp,seq)
            self.label = label
            self.seqHeader = seqHeader
            self.importindex = index
            self.orgLen=len(seq)
        else:
            print("Not a DNA SEQUENCE, last failed at char index "+str(failindex)+": "+str(seq[failindex]))

            
    def stringComptoPro(self,str2,dictKey):# simple method to count occurances of string match
        matches=""
        for i in range(len(self.seq)):
            if self.seq[i:i+len(str2)] == str2:
                matches+=dictKey
        return matches
        
    def convertfromKEY(self,convDict):# a dictionary lookup from KEY
        newProSeq=""
        for i in self.seq:
            if i in convDict:
                newProSeq+=str(convDict[i])+"\n"
        return newProSeq
    
    def convertfromVal(self,convDict):# a dictionary lookup for Value
        newProSeq=""        
        for i in range(len(self.seq)):
            for key, val in convDict.items():
                for vals in val:
                    if self.seq[i:i+len(vals)] == vals: 
                        newProSeq+= key
        return newProSeq
    def __repr__(self):
        if self.orgLen == len(self.seq):  # Checks if original Char length or if Modified
            retstr ="I'm DNASequence: "+self.label+"\n"
            retstr+="DataSequence length:" +str(self.orgLen)+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        else:
            retstr ="I'm DNASequence: "+self.label+"\n"
            retstr+="DataSequence orignal length:" +str(self.orgLen)+" but was modified to new Length:"+str(len(seq))+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq +"\n-------------------------------------------------------------"
        return retstr
                

#### child class 2 of 2 (Protein Sequence) allows linking from converted DNA sequence
specific string return

In [5]:
class ProSeq(Sequence): #Child sequence class to harness DNA sequence record
    def __init__(self,stamp,seq,label):
        super().__init__(stamp,seq)
        self.label = label
        self.DNAimportbit = False # a boolean set false later toggled by method linking to DNAseq
        self.DNAseqlabel=""
        self.importindex = 0
    
    def setFromDNA(self,genLabel,index):  #method to update generating sequence label and toggle import bit
        self.DNAseqlabel = genLabel
        self.DNAimportbit =True
        self.importindex = index
        
    
    def __repr__(self):
        retstr ="I'm Protein Sequence: "+self.label+"\n"
        if self.DNAimportbit == False:
            retstr+="DataSequence length:" +str(len(self.seq))+"\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        else:
            retstr+="DataSequence genrated from " +self.DNAseqlabel+" with a length of:" +str(len(self.seq))+"\n""\n"
            retstr+="______________________________________________\n"
            retstr+=self.seq+"\n-------------------------------------------------------------"
        return retstr

In [6]:
#supplied DNA translation 
aa_dict = {'M':['ATG'], 'F':['TTT', 'TTC'], 'L':['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], 'C':['TGT', 'TGC'], 'Y':['TAC', 'TAT'], 'W':['TGG'], 'P':['CCT', 'CCC', 'CCA', 'CCG'], 'H':['CAT', 'CAC'],
'Q':['CAA', 'CAG'], 'R':['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'I':['ATT', 'ATC', 'ATA'], 'T':['ACT', 'ACC', 'ACA', 'ACG'],
'N':['AAT', 'AAC'], 'K':['AAA', 'AAG'], 'S':['AGT', 'AGC', 'TCT', 'TCC', 'TCA', 'TCG'], 'V':['GTT', 'GTC', 'GTA', 'GTG'],
'A':['GCT', 'GCC', 'GCA', 'GCG'], 'D':['GAT', 'GAC'], 'E':['GAA', 'GAG'], 'G':['GGT', 'GGC', 'GGA', 'GGG'], '*':['TAA','TAG','TGA']}

#### main () I know that there are only 3 returning dictionary entries in supllied FASTA file, so I generate 3 sets of class objects.  

In [7]:
counter = 1
currentStamp = dt.datetime.now()
for name,seq in readMultilineFASTA("SampleData.fa"):
    if counter == 1:
        DNAtest1 = DNASeq(currentStamp,seq,name,"DNA1",counter)
        Protest1 = ProSeq(currentStamp,DNAtest1.convertfromVal(aa_dict),"ProfromDNA1")
        Protest1.setFromDNA(DNAtest1.label,counter*counter)
    if counter == 2:
        DNAtest2 = DNASeq(currentStamp,seq,name,"DNA2",counter)
        Protest2 = ProSeq(currentStamp,DNAtest2.convertfromVal(aa_dict),"ProfromDNA2")
        Protest2.setFromDNA(DNAtest2.label,counter*counter) 
    if counter == 3:
        DNAtest3 = DNASeq(currentStamp,seq,name,"DNA3",counter)
        Protest3 = ProSeq(currentStamp,DNAtest3.convertfromVal(aa_dict),"ProfromDNA3")
        Protest3.setFromDNA(DNAtest3.label,counter*counter)
    counter+=1

In [8]:
print(Protest2)
print(DNAtest2)

I'm Protein Sequence: ProfromDNA2
DataSequence genrated from DNA2 with a length of:1729

______________________________________________
MWGARVSLSLSLSPHISRALFSHTHILYTLSPHIFFSPLSHMWGAPLSPPQKRENI*SAQNTHTPQSVYTPHTPPLSLSHISLFFSQNTQKKNTPPPHIFFSRV*KKKNTHTPPQKKNISLSHTPLFFFSREKKKKNTRAPHISHIFFFSLSHTRVFFFSPHM*EKKKNM*DTPQRGEKKNTRENTPPHTLFSLFSLFSRVSRAPRVSRVSRGALCVLCAQSVFFSPREREKSAPRESARDTRGAQSV*ERDTPPQRGERESV*ENILCAPQSAQREKRGESAL*DTLSL*ESVSL*DM*ERGEREKKREREKSAQSALWGERDTRGDILWGEKRDTRGDTRERGESVFSQKREKSVFSHMWGGEKNISPLFSRDIL*EKSALCAQNI*KKSVLWGEREKREKKKRGGAQRDI*RGEKNTLSQKRGDISLL*DI*RGGEKKSV*SVWGGENTPPHIFLCAQRGGDISPL*ENTQKKSVFFLCARV*DTQSVFL*DTPQREREREKRGERERGALWGEREKKSAL*EKRERDTRVFSQKRGAQNIL*DISLL*NTQKSVL*KRGESVLCVFFLWGGVFL*DIYTHIFFFFFSPQNTL*DMCVFSRGERDIFLWGERDMWGERGAHIFFFLCVLWGEKNIFL*ERGERDTPQNIL*EKRERGVSHMCAPPQKNIL*RESAQKKKSALYISL*EKSARGAQRGEKREREKSVLCVCVLYMWGVFSHMWGEKREKKKKNTL*NM*DTHISQREKKNTQRGVSLCVYMWGVCAQNTPPQKNILWGEKNIL*DISLCAQSVFL*ESVSQNTPQKSAL*ESAHTLSPLFLWGGVLYMCV*ESVCAQSV*SAPLLYMCVCVSRAPHTRVFLW

In [9]:
print(Protest1)
print(DNAtest1)

I'm Protein Sequence: ProfromDNA1
DataSequence genrated from DNA1 with a length of:2944

______________________________________________
M*DMCAQSVSPRVWGALSPLCVCVCAQNMCVSLCARGARESAQRGVWGGGALLWGVCAPQNMWGGGERGVFFFSRVWGAHMCAPHTRESVCV*NIFFSPPPHIFLCAQKRGALLCVFFSRDM*EKRDMCVSQKRGALWGGARV*KKSVFLCALLCAQSVCVWGVYIFSPPLYM*DTRDI*NTPPRVLWGARGESVYM*EKNTQKKRGVCVSQRGAHTLSRDISPHTQNMWGEKSALSHTPL*ENI*NIYTHTQRGDIYTQRGENIFSHMCAL*RGAHIYISQSAQSVCVCVSLYTRVLWGDI*SV*ENIL*KNTRDM*ENISLWGAQNISPRDIFLWGEKRENI*RESVWGEKKSVLWGEKRGDI*KREKRGDI*KREKRGDI*KKKKRDISQKREKREKKKKRGDIYTHTPL*NMWGGGEKKKKRERGALSQKNIFSPHTPL*EREKSAQRDM*DTQRERGENIYIFSHISQRERGALCARGENTPHTLFFSQNTLSLSRVSPPHTLFSPHISL*NTQRENISHTHTPHIYTQRENTLCVLYI*NILYMCARDIL*DISHIFSLSRAPPLFFFSLFSPHILYISRESV*NTQKNISPLCVL*DI*SVCALLYTRGVSLYMWGVFSHTLFSRDISHIYMCV*ERDISLWGVFLCALFFFFSLLWGGVCVLWGDISQSVFFSPL*KSVWGVSLSPQSVL*NISRGDTLYTHIFLYTL*DTQRGVLYISLCAPQRGVFL*EKKRERERGGV*ENTLSLSPRESALLCALCALCVWGDIFFSLFSRV*KSVYTQSVL*DISPRVL*EKKRENTPRAPPLL*DILYTLCAPQNIYTPRVCALFFSLYISPLLCALCV*RDTLYTPPLC

In [10]:
print(DNAtest3)
print(Protest3)

I'm DNASequence: DNA3
DataSequence length:468
______________________________________________
ATGTCGTCGTCGTCGAAGATCACCCTGCAGAGCTCCGACGGCGAGTCGTTCGAGGTCGAGGAGGCGGTGGCGCTGGAATCACAGACCATTGGGCTCGTGATCGAGAACGACTGCGCCGACAACTGTATTCCTTTGCCCAACGTCACCAGCAAGATCTTGGCCATGGTCATCGAGTACTGCAAGAGGCACGTCGACGCCGCCAAGCTCGACGAGAAGATCTTCGAGGACGATCTCGAGTCCTGGGATCAGGCGTTCGTAAAGGTGGACCAGGCTACGCTGTTTGATCTCATTAACGCTGCAGCCTACTTGAACATCAAGAGCCTTTTGGACCTGACATGCCAGGCAGTTGCAGACATGATCAAGGGTAAGACTCCAGAAGAGATCAGAAAGACCTTCAACATCAAGGACGAGTTCACCCTGGAGGAAGAGGAGGAAGTTCGTCGTGAGAACCAGTGGGCGTTTGAATGA
-------------------------------------------------------------
I'm Protein Sequence: ProfromDNA3
DataSequence genrated from DNA3 with a length of:466

______________________________________________
MCVSRVSRVSRVSREKRDISHTPPLCAQRESALSPRDTRGARESVSRVFSRERGVSRERGERGARGVWGARALWGENISHTQRDTPHILWGGALSRV*DISRERENTRDTLCARAPRDTQNTLCVYIFSPLFLCAPPQNTRVSHTPQSAQKRDISLLWGAPHMWGVSHISRESVYTLCAQKRERGAHTRVSRDTRAPRAPQKSALSRDTREREKRDISLFSRERGDTRDISLSRESVSPLW

## I added a datetime stamp to seq class, and import index to child classes thinking that might aid in sorting, these instance variables where not used but were assigned